In [ ]:
using LinearAlgebra: norm, opnorm, eigen
using Plots 
using Pickle # for saving data. One can comment it

include("problem_instances.jl")
using .Problems

## Euclidean run


| id  | function  | n  |   |   |
|---|---|---|---|---|
|probl_1| policeman_and_burglar_matrix  |  500 |   |   |
|probl_2| nemirovski1  | 500  |   |   |
|probl_3| nemirovski2   | 500  |   |   |

The first problem is randomly generated. For the paper we used seed `sd="1"`. Set `sd="false"` otherwise. For the first run of code set `m, n, N` small

In [ ]:
# Choose dimensions and problem instance
m, n = 50, 50
batch_sizes = [1, 3, 5, 10]

probl = 1 # 1, 2 or 3
sd = "1"


if probl == 1
    A = Problems.policeman_and_burglar_matrix(n; seed=sd)
    filename = "save/probl_1e" 
elseif probl == 2
    A = Problems.nemirovski1(n)
    filename = "save/probl_2e" 
elseif probl == 3
    A = Problems.nemirovski2(n, 2)
    filename = "save/probl_3e"
elseif probl == 4
    A = Problems.randunif(m, n; seed=sd)
    filename = "save/probl_4e"
end

In [ ]:
# Compute different norms of A
max_norm = maximum(abs.(A))
l2_norm = norm(A, 2)
sp_norm = opnorm(A) # L
nnz = length(A[A .!= 0])
print("Operator norm of A is $(round(sp_norm, digits=1)),
Frobenius norm is $(round(l2_norm, digits=1)),
max norm is $(round(max_norm, digits=1))
nnz(A) is $nnz. Total elements in A: $(m*n)")

In [ ]:
x0 = ones(n)/n 
y0 = ones(m)/m

z0 = [x0; y0]
tol = 1e-8

In [ ]:
A

## Make quick plot

In [ ]:
gr()

# some hack to put markers less frequently. From https://github.com/JuliaPlots/Plots.jl/issues/2523#issuecomment-607090470
# now their frequency is controled by parameter `step`

@recipe function f(::Type{Val{:samplemarkers}}, x, y, z; step = 500)
    n = length(y)
    sx, sy = x[1:step:n], y[1:step:n]
    # add an empty series with the correct type for legend markers
    @series begin
        seriestype := :path
        markershape --> :auto
        x := [Inf]
        y := [Inf]
    end
    # add a series for the line
    @series begin
        primary := false # no legend entry
        markershape := :none # ensure no markers
        seriestype := :path
        seriescolor := get(plotattributes, :seriescolor, :auto)
        x := x
        y := y
    end
    # return  a series for the sampled markers
    primary := false
    seriestype := :scatter
    markershape --> :auto
    x := sx
    y := sy
end
### end of hack

#### Set number of epochs `N` and run the algorithms

In [ ]:
include("algorithms.jl")
projection(x) = proj_simplex1(x)
N = 10000 # max epoch

#### Define function for searching $\gamma$.

In [ ]:
# find γ that deliver the fastet convergency to the threshold
function find_γ(method::Function, b::Int, γ_arr::Array{Float64, 1}, thrs::Float64=1e-5)
    thrs_reached = false
    γ_best = γ_arr[1]
    res = method(b, γ_best)
    ind = findfirst(res[1] .<= thrs)
    iter_best = res[4][isnothing(ind) ? end : ind]
    for γ in γ_arr[2:end]
        res = method(b, γ)
        ind = findfirst(res[1] .<= thrs)
        if res[4][isnothing(ind) ? end : ind] < iter_best
            iter_best = res[4][isnothing(ind) ? end : ind]
            γ_best = γ
        end
    end
    return γ_best
end

#### Define function for drawing methods with different $\gamma$s.

In [ ]:
function draw_γ(method::Function, label::String, b::Int, γ_arr::Array{Float64, 1})
    γ = γ_arr[1]
    res = method(b, γ)
    ind = res[1][1:min(size(res[1])[1], size(res[4])[1])] .> 0
    display(plot(res[4][ind], res[1][ind], label=label*" γ:$γ", linewidth=2, marker=:utriangle, seriestype=:samplemarkers, step=45, yaxis=:log))
    for γ in γ_arr[2:end]
        IJulia.clear_output(true)
        res = method(b, γ)
        ind = res[1][1:min(size(res[1])[1], size(res[4])[1])] .> 0
        display(plot!(res[4][ind], res[1][ind], label=label*" γ:$γ", linewidth=2, marker=:utriangle, seriestype=:samplemarkers, step=45, yaxis=:log))
    end
end

#### Define function for drawing methods with different batch sizes.

In [ ]:
function draw_b(method::Function, label::String, γ::Float64, b_arr::Array{Int, 1})
    b = b_arr[1]
    res = method(b, γ)
    ind = res[1][1:min(size(res[1])[1], size(res[4])[1])] .> 0
    display(plot(res[4][ind], res[1][ind], label=label*" b:$b", linewidth=2, marker=:utriangle, seriestype=:samplemarkers, step=45, yaxis=:log))
    for b in b_arr[2:end]
        IJulia.clear_output(true)
        res = method(b, γ)
        ind = res[1][1:min(size(res[1])[1], size(res[4])[1])] .> 0
        display(plot!(res[4][ind], res[1][ind], label=label*" b:$b", linewidth=2, marker=:utriangle, seriestype=:samplemarkers, step=45, yaxis=:log))
    end
end

## Stochastic extragradient with variance reduction (loopless)

In [ ]:
function compute_eg_loopless(b_eg_loopless::Int, γ_eg_loopless::Float64=0.99, theor::Bool=false)
    K_eg_loopless = ceil(Int64, nnz/(m + n)/b_eg_loopless)
    p_eg_loopless = 1.0 / K_eg_loopless
    α_eg_loopless = 1 - p_eg_loopless
    L_eg_loopless = sp_norm
    
    τ_eg_loopless = sqrt(1-α_eg_loopless)/L_eg_loopless * γ_eg_loopless
    return stochExtraGradLoopless(A, projection, z0, τ_eg_loopless, α_eg_loopless, p_eg_loopless,
        b_eg_loopless, N, distr=true, tol=tol)
end

In [ ]:
find_γ(compute_eg_loopless, 1, [0.0001, 0.001, 0.01, 0.1, 0.3, 0.4, 0.5, 0.9, 0.99, 1.0])

In [ ]:
draw_γ(compute_eg_loopless, "EG-Loopless", 1, [0.0001, 0.001, 0.01, 0.1, 0.3, 0.4, 0.5, 0.9, 0.99, 1.0])

In [ ]:
draw_b(compute_eg_loopless, "EG-Loopless", 0.4, batch_sizes)

## Stochastic extragradient with variance reduction (looped)

In [ ]:
function compute_eg_looped(b_eg_looped::Int, γ_eg_looped::Float64=0.99, theor::Bool=false)
    K_eg_looped = ceil(Int64, nnz/(m + n)/b_eg_looped)
    p_eg_looped = 1.0 / K_eg_looped
    α_eg_looped = 1 - p_eg_looped
    L_eg_looped = sp_norm
    
    τ_eg_looped = sqrt(1-α_eg_looped)/L_eg_looped * γ_eg_looped
    return stochExtraGradLooped(A, projection, z0, τ_eg_looped, α_eg_looped, K_eg_looped, b_eg_looped, N, 
        distr=true, tol=tol)
end

In [ ]:
find_γ(compute_eg_looped, 1, [0.0001, 0.001, 0.01, 0.1, 0.3, 0.4, 0.5, 0.9, 0.99, 1.0])

In [ ]:
draw_γ(compute_eg_looped, "EG-Looped", 1, [0.0001, 0.001, 0.01, 0.1, 0.3, 0.4, 0.5, 0.9, 0.99, 1.0])

In [ ]:
draw_b(compute_eg_looped, "EG-Looped", 0.4, batch_sizes)

## Algorithm 2

In [ ]:
function compute_algorithm2(b_algorithm2::Int, c_2::Float64=1.3, theor::Bool=false)
    w0 = z0
    L = l2_norm
    γ_2 = min(1/16, b_algorithm2 / n)
    p_2 = γ_2
    α_2 = 1 / p_2
    if theor
        η_2 = c_2 * min(sqrt(α_2 * γ_2 * b_algorithm2) / 2 * L, 1 / (8 * L))
    else
        η_2 = c_2 / L
    end
    
    return stochAlgorithm2(A, projection, z0, w0, α_2, γ_2, η_2, p_2, b_algorithm2, N, distr=true, tol=tol)
end

In [ ]:
find_γ(compute_algorithm2, 1, [0.0001, 0.001, 0.01, 0.1, 0.3, 0.4, 0.5, 0.9, 0.99, 1.0])

In [ ]:
draw_γ(compute_algorithm2, "Algorithm2", 1, [0.0001, 0.001, 0.01, 0.1, 0.3, 0.4, 0.5, 0.9, 0.99, 1.0])

In [ ]:
draw_b(compute_algorithm2, "Algorithm2", 0.1, batch_sizes)

## Carmon et al.

In [ ]:
function compute_MPCarmon(b_mp_carmon::Int, step_mult::Float64=0.01, theor::Bool=false)
    alpha_ = l2_norm * sqrt((m + n)/(2 * m * n) / b_mp_carmon)
    η = alpha_ / (step_mult * l2_norm^2);
    
    return stochMPCarmon(A, projection, z0, alpha_, η, b_mp_carmon, N, distr=true, tol=tol)
end

In [ ]:
find_γ(compute_MPCarmon, 1, [0.0001, 0.001, 0.01, 0.1, 0.3, 0.4, 0.5, 0.9, 0.99, 1.0])

In [ ]:
draw_γ(compute_MPCarmon, "MP-Car", 1, [0.0001, 0.001, 0.01, 0.1, 0.3, 0.4, 0.5, 0.9, 0.99, 1.0])

In [ ]:
draw_b(compute_MPCarmon, "MP-Car", 0.0001, batch_sizes)

# Methods collection

In [ ]:
function plot_b(b::Int, γ_arr::Array{Float64, 1}, thrs::Float64=1e-5)
    methods = [compute_eg_loopless, compute_eg_looped, compute_algorithm2, compute_MPCarmon]
    #γ_arr = [0.0001, 0.001, 0.01, 0.1, 0.3, 0.4, 0.5, 0.9, 0.99, 1.0]
    labels = ["EG-Loopless b:$b", "EG-Looped b:$b", "Algorithm2 b:$b", "MP-Car b:$b"]
    appendix = ["EG-Loopless", "EG-Looped", "Algorithm2", "MPC"]
    for i in 1:length(methods)
        IJulia.clear_output(true)
        method = methods[i]
        label = labels[i]
        #γ = find_γ(method, b, γ_arr, thrs)
        γ = γ_arr[i]
        res = method(b, γ)
        ind = res[1][1:min(size(res[1])[1], size(res[4])[1])] .> 0
        if i == 1
            display(plot(res[4][ind], res[1][ind], label=label*" γ:$γ", linewidth=2, marker=:utriangle, seriestype=:samplemarkers, step=45, yaxis=:log))
        else
            display(plot!(res[4][ind], res[1][ind], label=label*" γ:$γ", linewidth=2, marker=:utriangle, seriestype=:samplemarkers, step=45, yaxis=:log))
        end
        Pickle.store(filename*"b$b"*appendix[i], res)
    end
    for i in 1:length(methods)
        IJulia.clear_output(true)
        method = methods[i]
        label = labels[i]
        γ = 1.
        res = method(b, γ, true)
        ind = res[1][1:min(size(res[1])[1], size(res[4])[1])] .> 0
        display(plot!(res[4][ind], res[1][ind], label=label*" γ:$γ", linewidth=2, marker=:utriangle, seriestype=:samplemarkers, step=45, yaxis=:log))
        Pickle.store(filename*"b$b"*appendix[i]*"orig", res)
    end
end

The best parameters for the first problem.

In [ ]:
params_probl1 = Dict(
1 => [1.0, 1.0, 0.1, 0.0001],
3 => [1.0, 1.0, 0.5, 0.0001],
5 => [1.0, 1.0, 0.5, 0.0001],
10 => [1.0, 1.0, 0.5, 0.01])

The best parameters for the second problem.

In [ ]:
params_probl2 = Dict(
1 => [1.0, 0.9, 0.99, 0.001],
3 => [1.0, 1.0, 1.0, 0.0001],
5 => [1.0, 1.0, 1.0, 0.01],
10 => [1.0, 0.9, 1.0, 0.0001])

The best parameters for the third problem.

In [ ]:
params_probl3 = Dict(
1 => [0.99, 0.9, 0.3, 0.0001],
3 => [0.99, 0.9, 0.99, 0.001],
5 => [1.0, 1.0, 0.4, 0.0001],
10 => [0.99, 1.0, 0.99, 0.01])

Timing is not important here, since we compare algorithms' performance with respect to arithmetic operations. And of course our implmentation is not so good.